<a href="https://colab.research.google.com/github/mJekal/PyTorch_study/blob/main/Ensembel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앙상블(Ensembel)

- 여러 개의 분류기(Classifier)를 생성하고 그 예측을 결합해서 예측 도출

앙상블의 특징
- 단일 모델의 약점을 다수의 모델들을 결합하여 보완
- 결정 트리의 단점이 과적합을 많은 분류기를 결합해 보완하고 장점인 직관적인 분류 기준은 강화됨

보팅(Voting)
- 일반적으로 서로 다른 알고리즘을 가진 분류기를 결합
- 보팅 유형
  - 하드 보팅
    - 다수결 원칙과 비슷
    - 다수 분류기가 결정한 예측값을 최종 결괏값으로 선정

  - 소프트 보팅
    - 각 분류기의 레이블 값 결정 확률을 평균해서 확률이 가장 높은 레이블 값을 최종 결괏값으로 선정
    - 일반적으로 소프트 보팅이 성능이 좋음

배깅(Bagging)
-  각각의 분류기가 모두 같은 유형의 알고리즘 기반
- 분류기마다 데이터 샘플링을 서로 다르게 가져가면서 학습
- 랜덤 포레스트가 대표적

부스팅(Boosting)
- 여러 분류기가 순차적으로 학습 수행
- 앞서 학습한 분류기가 예측이 틀린 데이터에 대해 다음 분류기는 올바르게 예측할 수 있게 가중치를 부여
- GBM, XgBoost, LightGBM 등
- 정형 데이터의 분류나 회귀에서는 GBM 부스팅 계열의 앙상블이 전반적으로 높은 예측 성능을 나타냄

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [5]:
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                    test_size=0.2 , random_state= 156)

vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

Voting 분류기 정확도: 0.9561


In [6]:
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))

LogisticRegression 정확도: 0.9474
KNeighborsClassifier 정확도: 0.9386
